<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Posibles bandas
directory = "./news/Fake.csv"

In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv(directory)
df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 23481


### 1 - Preprocesamiento

In [5]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [6]:
# Demos un vistazo
sentence_tokens[:2]

[['donald',
  'trump',
  'sends',
  'out',
  'embarrassing',
  'new',
  'year’s',
  'eve',
  'message',
  'this',
  'is',
  'disturbing'],
 ['drunk',
  'bragging',
  'trump',
  'staffer',
  'started',
  'russian',
  'collusion',
  'investigation']]

### 2 - Crear los vectores (word2vec)

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [8]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [9]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [10]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 23481


In [11]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 6734


### 3 - Entrenar el modelo generador

In [12]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 3113261.0
Loss after epoch 1: 2095223.5
Loss after epoch 2: 1962520.5
Loss after epoch 3: 1870859.0
Loss after epoch 4: 1765405.0
Loss after epoch 5: 1707793.0
Loss after epoch 6: 1653973.0
Loss after epoch 7: 1604886.0
Loss after epoch 8: 1537369.0
Loss after epoch 9: 1443158.0
Loss after epoch 10: 1404536.0
Loss after epoch 11: 1372356.0
Loss after epoch 12: 1340252.0
Loss after epoch 13: 1312860.0
Loss after epoch 14: 1288504.0
Loss after epoch 15: 1265902.0
Loss after epoch 16: 1251040.0
Loss after epoch 17: 1235744.0
Loss after epoch 18: 1227042.0
Loss after epoch 19: 1220938.0


(5421054, 7049000)

### 4 - Ensayar

In [13]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["headline"], topn=10)

[('editorial', 0.6316870450973511),
 ('spaces', 0.6303924918174744),
 ('twisted', 0.6198528409004211),
 ('journalism', 0.6168592572212219),
 ('patriotism', 0.6157949566841125),
 ('biker', 0.6094646453857422),
 ('savages', 0.6092650294303894),
 ('rhetoric”', 0.6087856888771057),
 ('waves', 0.6029744744300842),
 ('inflammatory', 0.5999173521995544)]

In [14]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["government"], topn=10)

[('into', 0.0500711165368557),
 ('hysterical', 0.025352858006954193),
 ('hilariously', 0.019717082381248474),
 ('late', 0.017960423603653908),
 ('while', 0.015972737222909927),
 ('calling', 0.013326420448720455),
 ('watch', 0.012024621479213238),
 ('audio', 0.011445322073996067),
 ('texas', 0.01050979271531105),
 ('book', 0.008429767563939095)]

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('seven', 0.6137977242469788),
 ('41', 0.6005611419677734),
 ('ten', 0.5680000185966492),
 ('five', 0.5525975227355957),
 ('53', 0.5524325370788574),
 ('wtf', 0.5480233430862427),
 ('76', 0.5334916710853577),
 ('83', 0.528716504573822),
 ('93', 0.5243951678276062),
 ('previously', 0.5237188339233398)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[('cash', 0.5081894397735596),
 ('laundering', 0.5021743178367615),
 ('bucks', 0.4899887442588806),
 ('toys', 0.4614701569080353),
 ('scheme', 0.45659297704696655)]

In [17]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

### 5 - Visualizar agrupación de vectores

In [18]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [19]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show()

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

Se realizan los embeddings utilzando un dataset de Fake news. En general, puede observarse que el resultado obtenido parece tener sentido. Se puede visualizar como los términos búscados en el espacio de embeddings devuelven similitudes altas con palabras relacionadas (Por ejemplo: números con números y money con cash o laundering). 

Es interesante notar como también el modelo relaciona nombres de personas con términos políticos (por ejemplo bernie-sanders-republican-democrat-party son todas palabras que se encuentran muy cercanas en el espacio de embeddings). Esto nuevamente muestra que el modelo está funcionando como se espera, términos similares son considerados de esta manera por el modelo. Sin embargo, también se puede remarcar que, en este caso, solo puede deducirse que bernie sanders es una persona ligada a la política, pero si se quisiera entender en particular su afinidad política, el embedding no es capaz de distinguirlo (asume ambos partidos como equidistantes). Según el objetivo que se tenga puede necesitarse entrenar al embedding de otra manera.